In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_violent2_FL_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
x = data.loc[:,:'five_year']
y = data['violent_two_year'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002,0.003,0.004,0.005]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         impurity=impurity, 
                         seed = 816)

#### GAM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate = learning_rate,
                       depth = depth,
                       estimators=estimators,
                       holdout_split=holdout_split,
                       seed=816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binh

In [4]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.6210160941139474, 0.06516184724978831)

In [5]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.6749315016529055, 0.04098275663439359)

## Lasso Stumps

In [6]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on=['person_id', 'screening_date'])

## split X & Y
X_stumps = data.loc[:,:'five_year1'].copy()
Y_stumps = data['violent_two_year'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_test_stumps.csv")
X_train_stumps = train_stumps.loc[:,:'five_year1'].copy()
X_test_stumps = test_stumps.loc[:,:'five_year1'].copy()
Y_train_stumps = train_stumps['violent_two_year'].values.copy()
Y_test_stumps = test_stumps['violent_two_year'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [8]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.09, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

22

In [9]:
single_stump_model['features']

['sex1',
 'age_at_current_charge30',
 'age_at_first_charge19',
 'p_incarceration1',
 'p_felassault1',
 'p_misdeassault1',
 'current_violence1',
 'p_misdemeanor5',
 'p_violence2',
 'p_violence3',
 'p_violence4',
 'p_violence5',
 'total_convictions1',
 'total_convictions7',
 'p_arrest1',
 'p_arrest5',
 'p_property5',
 'p_drug2',
 'p_drug4',
 'p_dui1',
 'one_year1',
 'three_year1']

#### Nested Cross Validation

In [10]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, \
                                              0.06, 0.07, 0.08, 0.09]},
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:46: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [11]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.09}, {'C': 0.09}, {'C': 0.08}, {'C': 0.09}, {'C': 0.09}],
 0.66142769456366,
 0.04660005141449648)

#### best stump model

In [12]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.09, 
                                      columns=cols, 
                                      seed=816)
best_stump_model['test_auc']

0.6646531886916502

## RiskSLIM -- Unconstrained

In [13]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.06, 
                                      columns=cols, 
                                      seed=816)
len(single_stump_model['features'])

17

In [14]:
selected_features = single_stump_model['features']

In [15]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['violent_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['violent_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0
    
sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [16]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator=indicator,
                                       y_label='violent_two_year', 
                                       max_coef=5,
                                       max_coef_number=5, 
                                       max_runtime=1000, 
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/20/19 @ 10:20 PM | 223 rows in lookup table
11/20/19 @ 10:20 PM | ------------------------------------------------------------
11/20/19 @ 10:20 PM | runnning initialization procedure
11/20/19 @ 10:20 PM | ------------------------------------------------------------
11/20/19 @ 10:20 PM | CPA produced 2 cuts
11/20/19 @ 10:20 PM | running naive rounding on 4 solutions
11/20/19 @ 10:20 PM | best objective value: 0.6931
11/20/19 @ 10:20 PM | rounding produced 4 integer solutions
11/20/19 @ 10:20 PM | best objective value is 0.6931
11/20/19 @ 10:20 PM | running sequential rounding on 4 solutions
11/20/19 @ 10:20 PM | best objective value: 0.6931
11/20/19 @ 10:20 PM | sequential rounding produced 0 integer solutions
11/20/19 @ 10:20 PM | polishing 4 solutions
11/20/19 @ 10:20 PM | best objective value: 0.6931
11/20/19 @ 10:20 PM | polishing produced 4 integer solutions
11/20/19 @ 10:20 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 10:20 PM | adding 304 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5038.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 116985 19475        0.4699     5        0.4712        0.4646   674095    1.41%          rho_11 U 116985 116984     38
 119818 19382        cutoff              0.4712        0.4648   685566    1.36%          rho_10 U 119818 111421     23
Elapsed time = 36.64 sec. (40870.64 ticks, tree = 6.09 MB, solutions = 12)
 122657 19186        0.4650    12        0.4712        0.4650   697009    1.32%          rho_10 U 122657 114241     22
 125490 19042        0.4657    13        0.4712        0.4652   708089    1.27%          rho_14 U 125490  68629     21
 128325 18805        cutoff              0.4712        0.4654   719114    1.23%          rho_15 U 128325 128324     34
 131165 18590        0.4688    11        0.4712        0.4656   730078    1.19%           rho_9 D 131165 131164     22
 133930 18284        cutoff              0.4712        0.4658   740726    1.14%           rho_6 U 133930 133928     22
 136780 18021        0.4681     7        0.4712        0.4660   751100    1.10%          rho

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 10:21 PM | adding 270 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5020.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  95928 31742        0.4653    17        0.4792        0.4626   713877    3.45%           rho_0 D  95928  95927     21
  98013 32271        0.4714    10        0.4792        0.4628   725009    3.41%          rho_10 D  98013  98012     31
 100116 32812        0.4667    13        0.4792        0.4630   736089    3.38%          rho_16 D 100116 100115     22
 102187 33242        0.4672    13        0.4792        0.4632   747064    3.34%           rho_6 D 102187  60060     37
 104286 33691        cutoff              0.4792        0.4634   758411    3.30%           rho_8 U 104286 104285     27
 106373 34158        0.4724    12        0.4792        0.4635   769175    3.27%          rho_12 N 106373  81502     20
 108375 34553        cutoff              0.4792        0.4637   779949    3.24%           rho_9 U 108375  20601     29
 110396 34928        cutoff              0.4792        0.4638   790718    3.21%          rho_11 D 110396 109074     33
 112439 35318        cutoff              0.4792 

...

CPXPARAM_RandomSeed                              0
CPXPARAM_TimeLimit                               1000
CPXPARAM_MIP_Tolerances_LowerCutoff              0.46866380383060319
CPXPARAM_MIP_Tolerances_UpperCutoff              0.52489851616271477


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:26 PM | adding 282 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5249.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  79267 27458        cutoff              0.4890        0.4687   573440    4.16%         alpha_4 N  79267  79266     21
  81053 28063        0.4702     2        0.4890        0.4687   585795    4.16%          rho_17 U  81053  81052     24
  82797 28693        0.4744     6        0.4890        0.4687   597359    4.16%           rho_2 U  82797  82795     22
Elapsed time = 38.27 sec. (40457.82 ticks, tree = 8.92 MB, solutions = 15)
  84698 29404        0.4721     8        0.4890        0.4687   608169    4.16%          rho_11 D  84698  84697     27
  86430 30260        0.4822     6        0.4890        0.4687   619540    4.16%         alpha_5 D  86430  86429     33
  88013 30871        0.4857     1        0.4890        0.4687   631517    4.16%           rho_4 U  88013  88012     33
  89717 31577        0.4796    13        0.4890        0.4687   643523    4.16%           rho_0 D  89717  89716     26
  91233 32220        0.4704    10        0.4890        0.4687   655561    4.16%          rho

 237419 55941        0.4820    13        0.4886        0.4757  1426122    2.64%           rho_7 U 237419  12489     16
 243438 56262        0.4842     9        0.4886        0.4760  1456279    2.59%           rho_9 D 243438 243436     30
 249436 56511        0.4830    14        0.4886        0.4762  1485814    2.54%          rho_17 U 249436  63113     35
 255493 56755        cutoff              0.4886        0.4765  1515607    2.49%           rho_7 D 255493 157090     24
 261577 56828        0.4874     5        0.4886        0.4767  1545423    2.43%           rho_5 D 261577 261576     34
 267565 56862        cutoff              0.4886        0.4769  1574327    2.38%           rho_7 D 267565 267563     27
 273490 56916        0.4774    15        0.4886        0.4772  1602685    2.34%           rho_9 D 273490 145247     36
 279453 56754        0.4774    10        0.4886        0.4774  1630849    2.28%          rho_11 D 279453 272055     28
 285410 56732        cutoff              0.4886 

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [17]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.6555031814080625, 0.6502623342775424)

### Single RiskSLIM Model -- Unconstrained

In [18]:
selected_features = ["violent_two_year"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [19]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'violent_two_year',
    'sample_weights': sample_weights
}

In [20]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/20/19 @ 11:30 PM | 223 rows in lookup table
11/20/19 @ 11:30 PM | ------------------------------------------------------------
11/20/19 @ 11:30 PM | runnning initialization procedure
11/20/19 @ 11:30 PM | ------------------------------------------------------------
11/20/19 @ 11:30 PM | CPA produced 2 cuts
11/20/19 @ 11:30 PM | running naive rounding on 5 solutions
11/20/19 @ 11:30 PM | best objective value: 0.6931
11/20/19 @ 11:30 PM | rounding produced 5 integer solutions
11/20/19 @ 11:30 PM | best objective value is 0.6931
11/20/19 @ 11:30 PM | running sequential rounding on 5 solutions
11/20/19 @ 11:30 PM | best objective value: 0.6931
11/20/19 @ 11:30 PM | sequential rounding produced 0 integer solutions
11/20/19 @ 11:30 PM | polishing 5 solutions
11/20/19 @ 11:30 PM | best objective value: 0.6931
11/20/19 @ 11:30 PM | polishing produced 5 integer solutions
11/20/19 @ 11:30 PM | initialization produced 9 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:30 PM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5086.
Tried aggregator 1 time.
Reduced MIP has 36 rows, 38 columns, and 105 nonzeros.
Reduced MIP has 17 binaries, 19 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

  95159 28554        0.4633    15        0.4810        0.4633   689210    3.68%           rho_0 U  95159  95144     37
Elapsed time = 31.59 sec. (39054.31 ticks, tree = 8.44 MB, solutions = 14)
  97067 29368        0.4760    11        0.4810        0.4633   704304    3.68%           rho_2 D  97067  97066     29
  99207 30253        0.4658    17        0.4810        0.4633   718502    3.68%           rho_8 D  99207  99182     26
 101078 31157        0.4669    19        0.4810        0.4633   733248    3.68%          rho_12 D 101078 101077     33
 103107 32093        0.4792    16        0.4810        0.4633   747858    3.68%          rho_12 D 103107 103105     29
 104730 32571        0.4764    18        0.4810        0.4633   762176    3.68%          rho_16 N 104730  76179     16
 106431 33246        0.4670    23        0.4810        0.4633   777499    3.68%          rho_13 U 106431 105218      9
 108248 33903        0.4633    15        0.4810        0.4633   791972    3.68%          rho

 299505 26296        cutoff              0.4810        0.4759  1778146    1.06%           rho_7 D 299505 265036     25
 306779 23302        0.4790    14        0.4810        0.4765  1808130    0.94%          rho_14 U 306779 284162     25
 314122 19980        cutoff              0.4810        0.4772  1836644    0.80%           rho_9 D 314122 314121     28
 321351 16199        cutoff              0.4810        0.4778  1864089    0.66%           rho_6 U 321351 160239     25
 328537 11867        cutoff              0.4810        0.4786  1889219    0.50%          rho_17 D 328537  85009     39
 335745  6890        cutoff              0.4810        0.4795  1911666    0.32%           rho_2 U 335745 263367     51
 342881  1164        cutoff              0.4810        0.4807  1930549    0.07%          rho_15 D 342881 231746     34
Elapsed time = 113.17 sec. (165922.89 ticks, tree = 3.02 MB, solutions = 14)

User cuts applied:  802

Root node processing (before b&c):
  Real time             =    

In [21]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [22]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.6920461879987483

In [23]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.6494904667981591

## RiskSLIM -- Constrained

In [24]:
## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.1,  
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

23

In [25]:
selected_features = single_stump_model['features']

In [26]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['violent_two_year', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['violent_two_year', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0
    
sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [27]:
riskslim_cs_summary = slim.risk_nested_cv_constrain(X=sub_X, 
                                                    Y=sub_Y, 
                                                    indicator=indicator,
                                                    y_label='violent_two_year', 
                                                    max_coef=5,
                                                    max_coef_number=5, 
                                                    max_runtime=1000, 
                                                    max_offset=100,
                                                    c=1e-6, 
                                                    seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/20/19 @ 11:32 PM | 283 rows in lookup table
11/20/19 @ 11:32 PM | ------------------------------------------------------------
11/20/19 @ 11:32 PM | runnning initialization procedure
11/20/19 @ 11:32 PM | ------------------------------------------------------------
11/20/19 @ 11:32 PM | CPA produced 2 cuts
11/20/19 @ 11:32 PM | running naive rounding on 13 solutions
11/20/19 @ 11:32 PM | best objective value: 0.4954
11/20/19 @ 11:32 PM | rounding produced 5 integer solutions
11/20/19 @ 11:32 PM | best objective value is 0.4882
11/20/19 @ 11:32 PM | running sequential rounding on 13 solutions
11/20/19 @ 11:32 PM | best objective value: 0.4954
11/20/19 @ 11:32 PM | sequential rounding produced 6 integer solutions
11/20/19 @ 11:32 PM | best objective value: 0.4843
11/20/19 @ 11:32 PM | polishing 11 solutions
11/20/19 @ 11:32 PM | best objective value: 0.4843
11/20/19 @ 11:32 PM | polishing produced 4 integer solutions
11/20/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:32 PM | adding 234 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4843.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:32 PM | adding 237 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4837.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:32 PM | adding 219 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4961.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:32 PM | adding 212 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4957.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:35 PM | adding 216 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4934.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:35 PM | adding 242 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4892.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:36 PM | adding 218 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.5053.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:36 PM | adding 240 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4916.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [28]:
np.mean(riskslim_cs_summary['validation_auc']), np.mean(riskslim_cs_summary['test_auc'])

(0.6562126265682405, 0.6490079408043652)

### Single RiskSLIM Model -- Constrained

In [29]:
selected_features = ["violent_two_year"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [30]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'violent_two_year',
    'sample_weights': sample_weights
}

In [31]:
model_info, mip_info, lcpa_info = slim.risk_slim_constrain(new_train_data, 
                                                           max_coefficient=5, 
                                                           max_L0_value=5, 
                                                           c0_value=1e-6, 
                                                           max_offset=100, 
                                                           max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/20/19 @ 11:38 PM | 283 rows in lookup table
11/20/19 @ 11:38 PM | ------------------------------------------------------------
11/20/19 @ 11:38 PM | runnning initialization procedure
11/20/19 @ 11:38 PM | ------------------------------------------------------------
11/20/19 @ 11:38 PM | CPA produced 2 cuts
11/20/19 @ 11:38 PM | running naive rounding on 14 solutions
11/20/19 @ 11:38 PM | best objective value: 0.5182
11/20/19 @ 11:38 PM | rounding produced 5 integer solutions
11/20/19 @ 11:38 PM | best objective value is 0.5347
11/20/19 @ 11:38 PM | running sequential rounding on 14 solutions
11/20/19 @ 11:38 PM | best objective value: 0.5182
11/20/19 @ 11:38 PM | sequential rounding produced 6 integer solutions
11/20/19 @ 11:38 PM | best objective value: 0.4931
11/20/19 @ 11:38 PM | polishing 11 solutions
11/20/19 @ 11:38 PM | best objective value: 0.4931
11/20/19 @ 11:38 PM | polishing produced 5 integer solutions
11/20/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/20/19 @ 11:38 PM | adding 242 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.4921.
Tried aggregator 1 time.
Reduced MIP has 25 rows, 50 columns, and 95 nonzeros.
Reduced MIP has 23 binaries, 25 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.05 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [32]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [33]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.6920461879987483

In [34]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.6494904667981591

### Arnold PSA

In [35]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['violent_two_year'].values

In [36]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
arnold_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    arnold_auc.append(roc_auc_score(test_y, test_x['arnold_nca'].values))
    
    ################################
    ## race_auc
    try:
        arnold_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['arnold_nca'],
                                                   labels = test_y)
        arnold_race_auc_final = arnold_race_auc.assign(fold_num = [i]*arnold_race_auc['Attribute'].count())
        race_auc.append(arnold_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    arnold_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['arnold_nca'],
                                                                         labels = test_y)
    arnold_condition_pn_final = arnold_condition_pn.assign(fold_num = [i]*arnold_condition_pn['Attribute'].count())
    condition_pn.append(arnold_condition_pn_final)
    
    ################################
    ## no condition pn
    arnold_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['arnold_nca'],
                                                                labels = test_y)
    arnold_no_condition_pn_final = arnold_no_condition_pn.assign(fold_num = [i]*arnold_no_condition_pn['Attribute'].count())
    no_condition_pn.append(arnold_no_condition_pn_final)
    
    i += 1
    
## race_auc
arnold_race_auc = []
try:
    arnold_race_auc = pd.concat(race_auc, ignore_index=True)
    arnold_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    arnold_race_auc = arnold_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
arnold_condition_pn = pd.concat(condition_pn, ignore_index=True)
arnold_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_condition_pn = arnold_condition_pn.reset_index(drop=True)

## no_condition_pn
arnold_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
arnold_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_no_condition_pn = arnold_no_condition_pn.reset_index(drop=True)

### Single Arnold PSA

In [37]:
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold_test.csv")
X = test_data['arnold_nvca_raw'].values
Y = test_data['violent_two_year'].values
roc_auc_score(Y, X)

0.645155325443787

## Compass

In [38]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['Risk of Violence_decile_score', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['violent_two_year'].values

In [39]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
compas_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    compas_auc.append(roc_auc_score(test_y, test_x['Risk of Violence_decile_score'].values))
    
    ################################
    ## race_auc
    try:
        compas_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['Risk of Violence_decile_score'],
                                                   labels = test_y)
        compas_race_auc_final = compas_race_auc.assign(fold_num = [i]*compas_race_auc['Attribute'].count())
        race_auc.append(compas_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    compas_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['Risk of Violence_decile_score'],
                                                                         labels = test_y)
    compas_condition_pn_final = compas_condition_pn.assign(fold_num = [i]*compas_condition_pn['Attribute'].count())
    condition_pn.append(compas_condition_pn_final)
    
    ################################
    ## no condition pn
    compas_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['Risk of Violence_decile_score'],
                                                                labels = test_y)
    compas_no_condition_pn_final = compas_no_condition_pn.assign(fold_num = [i]*compas_no_condition_pn['Attribute'].count())
    no_condition_pn.append(compas_no_condition_pn_final)
    
    i += 1
    
## race_auc
compas_race_auc = []
try:  
    compas_race_auc = pd.concat(race_auc, ignore_index=True)
    compas_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    compas_race_auc = compas_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
compas_condition_pn = pd.concat(condition_pn, ignore_index=True)
compas_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_condition_pn = compas_condition_pn.reset_index(drop=True)

## no_condition_pn
compas_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
compas_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_no_condition_pn = compas_no_condition_pn.reset_index(drop=True)

### Results

In [40]:
#### save results
summary_violent2_FL_interpret = {"CART": cart_summary,
                                 "EBM": ebm_summary, 
                                 'Lasso Stumps': stump_summary, 
                                 'RiskSLIM': riskslim_summary, 
                                 'RiskSLIM-Constrained': riskslim_cs_summary, 
                                 'Arnold PSA': arnold_auc, 
                                 'Compas': compas_auc}
%store summary_violent2_FL_interpret

Stored 'summary_violent2_FL_interpret' (dict)


In [41]:
results = [["CART", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["EBM", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["Lasso Stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['RiskSLIM', np.mean(riskslim_summary['test_auc'])],
           ['RiskSLIM-Constrained', np.mean(riskslim_cs_summary['test_auc'])],
           ['Arnold PSA', round(np.mean(arnold_auc), 3)], 
           ['Compas', round(np.mean(compas_auc), 3)]]
results

[['CART', 0.6210160941139474, 0.06516184724978831],
 ['EBM', 0.6749315016529055, 0.04098275663439359],
 ['Lasso Stumps', 0.66142769456366, 0.04660005141449648],
 ['RiskSLIM', 0.6502623342775424],
 ['RiskSLIM-Constrained', 0.6490079408043652],
 ['Arnold PSA', 0.628],
 ['Compas', 0.639]]

In [42]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc']),
       np.mean(riskslim_cs_summary['test_auc']),]

In [43]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/model results/Interpretable Models/Two Year/"
results = [["Violent", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            np.str(round(np.mean(riskslim_cs_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_cs_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold_auc), 3)) + " (" + np.str(round(np.std(arnold_auc),3)) + ")", 
            np.str(round(np.mean(compas_auc), 3)) + " (" + np.str(round(np.std(compas_auc),3)) + ")"]]
with open(path + 'Interpretable Models Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Confusion Matrix

In [44]:
cart_confusion = cart_summary['confusion_matrix_stats']
ebm_confusion = ebm_summary['confusion_matrix_stats']
riskslim_confusion = riskslim_summary['confusion_matrix_stats']
riskslim_cs_confusion = riskslim_cs_summary['confusion_matrix_stats']
stumps_confusion = stump_summary['confusion_matrix_stats']

cart_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/violent/cart_confusion.csv', index=None,header=True)
ebm_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/violent/ebm_confusion.csv', index=None,header=True)
riskslim_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/violent/riskslim_confusion.csv', index=None,header=True)
riskslim_cs_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/violent/riskslim_cs_confusion.csv', index=None,header=True)
stumps_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/two-year/violent/stumps_confusion.csv', index=None,header=True)

### Calibration

In [45]:
cart_calibration = cart_summary['calibration_stats']
ebm_calibration = ebm_summary['calibration_stats']
riskslim_calibration = riskslim_summary['calibration_stats']
riskslim_cs_calibration = riskslim_cs_summary['calibration_stats']
stumps_calibration = stump_summary['calibration_stats']

cart_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/violent/cart_calibration.csv', index=None,header=True)
ebm_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/violent/ebm_calibration.csv', index=None,header=True)
riskslim_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/violent/riskslim_calibration.csv', index=None,header=True)
riskslim_cs_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/violent/riskslim_cs_calibration.csv', index=None,header=True)
stumps_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/two-year/violent/stumps_calibration.csv', index=None,header=True)

### Race AUC

In [46]:
cart_race_auc = cart_summary['race_auc']
ebm_race_auc = ebm_summary['race_auc']
riskslim_race_auc = riskslim_summary['race_auc']
riskslim_cs_race_auc = riskslim_cs_summary['race_auc']
stumps_race_auc = stump_summary['race_auc']

## save results
cart_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/cart_race_auc.csv', index=None,header=True)
ebm_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/ebm_race_auc.csv', index=None,header=True)
riskslim_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/riskslim_race_auc.csv', index=None,header=True)
riskslim_cs_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/riskslim_cs_race_auc.csv', index=None,header=True)
stumps_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/stumps_race_auc.csv', index=None,header=True)
arnold_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/arnold_race_auc.csv', index=None,header=True)
compas_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/two-year/violent/compas_race_auc.csv', index=None,header=True)

### Condition PN

In [47]:
cart_condition_pn = cart_summary['condition_pn']
ebm_condition_pn = ebm_summary['condition_pn']
riskslim_condition_pn = riskslim_summary['condition_pn']
riskslim_cs_condition_pn = riskslim_cs_summary['condition_pn']
stumps_condition_pn = stump_summary['condition_pn']

## save results
cart_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/cart_condition_pn.csv', index=None,header=True)
ebm_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/ebm_condition_pn.csv', index=None,header=True)
riskslim_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/riskslim_condition_pn.csv', index=None,header=True)
riskslim_cs_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/riskslim_cs_condition_pn.csv', index=None,header=True)
stumps_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/stumps_condition_pn.csv', index=None,header=True)
arnold_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/arnold_condition_pn.csv', index=None,header=True)
compas_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/two-year/violent/compas_condition_pn.csv', index=None,header=True)

### No Condition PN

In [48]:
cart_no_condition_pn = cart_summary['no_condition_pn']
ebm_no_condition_pn = ebm_summary['no_condition_pn']
riskslim_no_condition_pn = riskslim_summary['no_condition_pn']
riskslim_cs_no_condition_pn = riskslim_cs_summary['no_condition_pn']
stumps_no_condition_pn = stump_summary['no_condition_pn']

## save results
cart_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/cart_no_condition_pn.csv', index=None,header=True)
ebm_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/ebm_no_condition_pn.csv', index=None,header=True)
riskslim_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/riskslim_no_condition_pn.csv', index=None,header=True)
riskslim_cs_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/riskslim_cs_no_condition_pn.csv', index=None,header=True)
stumps_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/stumps_no_condition_pn.csv', index=None,header=True)
arnold_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/arnold_no_condition_pn.csv', index=None,header=True)
compas_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/two-year/violent/compas_no_condition_pn.csv', index=None,header=True)